In [22]:
import os
import pickle
import datasets
import random
import transformers
traindata = datasets.load_dataset("/home/workspace/code/git/FlatQuant_mlm/datasets/wikitext", split="validation")
tokenizer = transformers.AutoTokenizer.from_pretrained("/home/workspace/model/MiniCPM-V-1B-sft-v2-1B", trust_remote_code=True)
trainenc = tokenizer("\n\n".join(traindata['text'][:1000]), return_tensors='pt')
seqlen = 2048
# random.seed(seed)
trainloader = []
for _ in range(3):
    i = random.randint(0, trainenc.input_ids.shape[1] - seqlen - 1)
    j = i + seqlen
    print("i",i)
    print("j",j)
    inp = trainenc.input_ids
    tar = inp.clone()
    tar[:, :-1] = -100
    trainloader.append((inp, tar))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


i 9430
j 11478
i 35583
j 37631
i 29072
j 31120


In [27]:
model = transformers.AutoModel.from_pretrained("/home/workspace/model/minicpm-v-2_6", 
                                                trust_remote_code=True,
                                                attn_implementation='sdpa', 
                                                torch_dtype='auto') 

Loading checkpoint shards: 100%|██████████| 4/4 [00:27<00:00,  6.94s/it]


In [1]:
#模型下载
from modelscope import snapshot_download
model_dir = snapshot_download('OpenBMB/MiniCPM-1B-sft-llama-format')

/opt/conda/envs/lmquant/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [29]:
model.processor

In [27]:
import os
import pickle
import datasets
import random
import transformers
from PIL import Image
import numpy as np
def get_mme(nsamples, seed, seqlen):
    import torch.nn.functional as F
    dataset = datasets.load_from_disk("/home/workspace/dataset/ScienceQA-2")["train"]
    dataset = dataset.shuffle(seed=seed)
    processor = transformers.AutoProcessor.from_pretrained("/home/workspace/model/MiniCPM-V-1B-sft-v2-1B", trust_remote_code=True)
    rng = random.Random(42)
    samples, num_tokens = [], 0
    prompts_lists = []
    input_images_lists = []
    for index, _data in enumerate(dataset):
        promt = _data["question"]
        image_file = _data["image"]
        image = np.array(image_file)
        if image_file is None:
            nsamples = nsamples+1
            break
        msgs = [{'role': 'user', 'content': "(<image>./</image>)\n"+ promt}]
        prompts_lists.append(processor.tokenizer.apply_chat_template(msgs, tokenize=False, add_generation_prompt=True))
        input_images_lists.append([image])
        if index >= nsamples:
            break
        inputs = processor(
            prompts_lists,
            input_images_lists,
            return_tensors="pt",
            max_length=8192
        )["input_ids"]
    # return inputs
    trainloader = []
    
    for i in range(inputs.size(0)):  # tensor.size(0) == 33
        inp = inputs.select(0, i).unsqueeze(0)  # 获取第 i 行并增加一个维度
        pad_size = seqlen - inp.size(1)
        # 在右侧填充，左边填充 0，右边填充 pad_size 个值
        # inp = F.pad(inp, (pad_size,0), "constant", 0)
        tar = inp.clone()
        tar[:, :-1] = -100
        trainloader.append((inp, tar))

    return trainloader

In [3]:
import os
import pickle
import datasets
import random
import transformers
from PIL import Image
import numpy as np
import torch
def get_mme(nsamples, seed, seqlen):
    import torch.nn.functional as F
    dataset = datasets.load_from_disk("/home/workspace/dataset/MME")["test"]
    dataset = dataset.shuffle(seed=seed)
    processor = transformers.AutoProcessor.from_pretrained("/home/workspace/model/MiniCPM-Llama3-V-2_5", trust_remote_code=True)
    rng = random.Random(42)
    samples, num_tokens = [], 0
    inputs_list = []
    for index, _data in enumerate(dataset):
        promt = _data["question"]
        image = _data["image"]
        msgs = [{'role': 'user', 'content': "(<image>./</image>)\n"+ promt}]
        prompts = processor.tokenizer.apply_chat_template(msgs, tokenize=False, add_generation_prompt=True)
        if index >= nsamples:
            break

        inputs = processor(
            prompts,
            image,
            return_tensors="pt",
            max_length=8192
        )["input_ids"]
    
        inputs_list.append(inputs)

    trainloader = []
    for i in inputs_list:  # tensor.size(0) == 33
        inp = inputs
        pad_size = seqlen - inp.size(1)
        # 在右侧填充，左边填充 0，右边填充 pad_size 个值
        inp = F.pad(inp, (pad_size,0), "constant", 0)
        tar = inp.clone()
        tar[:, :-1] = -100
        trainloader.append((inp, tar))

    return trainloader


In [51]:
import os
import pickle
import datasets
import random
import transformers
from PIL import Image
import numpy as np
import torch
def get_mme(nsamples, seed, seqlen):
    import torch.nn.functional as F
    dataset = datasets.load_from_disk("/home/workspace/dataset/ScienceQA-2")["train"]
    dataset = dataset.shuffle(seed=seed)
    processor = transformers.AutoProcessor.from_pretrained("/home/workspace/model/MiniCPM-Llama3-V-2_5", trust_remote_code=True)
    rng = random.Random(42)
    samples, num_tokens = [], 0
    inputs_list = []
    for index, _data in enumerate(dataset):
        promt = _data["question"]
        image = _data["image"]
        # image = np.array(image_file)
        if image is None:
            nsamples = nsamples + 1
            continue
        msgs = [{'role': 'user', 'content': "(<image>./</image>)\n"+ promt}]
        prompts = processor.tokenizer.apply_chat_template(msgs, tokenize=False, add_generation_prompt=True)
        if index >= nsamples:
            break

        inputs = processor(
            prompts,
            image,
            return_tensors="pt",
            max_length=8192
        )["input_ids"]
    
        inputs_list.append(inputs)

    trainloader = []
    for i in inputs_list:  # tensor.size(0) == 33
        inp = inputs
        pad_size = seqlen - inp.size(1)
        # 在右侧填充，左边填充 0，右边填充 pad_size 个值
        inp = F.pad(inp, (pad_size,0), "constant", 0)
        tar = inp.clone()
        tar[:, :-1] = -100
        trainloader.append((inp, tar))

    return trainloader


In [4]:
a = get_mme(32, 0, 1024)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
for i in a:
    print(a[0][1].shape)

torch.Size([1, 1024])
torch.Size([1, 1024])
torch.Size([1, 1024])
torch.Size([1, 1024])
torch.Size([1, 1024])
torch.Size([1, 1024])
torch.Size([1, 1024])
torch.Size([1, 1024])
torch.Size([1, 1024])
torch.Size([1, 1024])
torch.Size([1, 1024])
torch.Size([1, 1024])
torch.Size([1, 1024])
torch.Size([1, 1024])
torch.Size([1, 1024])
torch.Size([1, 1024])
torch.Size([1, 1024])
torch.Size([1, 1024])
torch.Size([1, 1024])
torch.Size([1, 1024])
torch.Size([1, 1024])
torch.Size([1, 1024])
torch.Size([1, 1024])
torch.Size([1, 1024])
torch.Size([1, 1024])
torch.Size([1, 1024])
torch.Size([1, 1024])
torch.Size([1, 1024])
torch.Size([1, 1024])
torch.Size([1, 1024])
torch.Size([1, 1024])
torch.Size([1, 1024])
